In [1]:
import torch
import torch.nn as nn
import lightning
from lightning.pytorch.callbacks import ModelCheckpoint
import yaml

from data.era5 import ERA5Dataset, gen_bgen
from models.autoencoder import Autoencoder
from models.losses import RecKLDiscriminatorLoss

In [2]:
with open("configs/autoencoder_kl_f16_attention.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

loss = RecKLDiscriminatorLoss(config=config)

device = "cpu"

vae = Autoencoder.load_from_checkpoint("checkpoints/vae-kl-f16-1440x720-69c-attention--step=31000.ckpt", config=config, strict=False).to(device)
vae.eval()

c:\Users\hendr\Desktop\3d-vae\venv\Lib\site-packages\lightning\pytorch\utilities\migration\utils.py:56: The loaded checkpoint was produced with Lightning v2.5.1, which is newer than your current Lightning version: v2.5.0.post0
c:\Users\hendr\Desktop\3d-vae\venv\Lib\site-packages\lightning\pytorch\core\saving.py:191: Found keys that are in the model state dict but not in the checkpoint: ['loss_model.discriminator.block1.0.weight', 'loss_model.discriminator.block1.0.bias', 'loss_model.discriminator.block2.0.weight', 'loss_model.discriminator.block2.0.bias', 'loss_model.discriminator.block2.1.weight', 'loss_model.discriminator.block2.1.bias', 'loss_model.discriminator.block2.1.running_mean', 'loss_model.discriminator.block2.1.running_var', 'loss_model.discriminator.block3.0.weight', 'loss_model.discriminator.block3.0.bias', 'loss_model.discriminator.block3.1.weight', 'loss_model.discriminator.block3.1.bias', 'loss_model.discriminator.block3.1.running_mean', 'loss_model.discriminator.block

Autoencoder(
  (encoder): Encoder(
    (conv_in): Conv2d(69, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownBlock(
        (layers): ModuleList(
          (0): Sequential(
            (0): GroupNorm(32, 128, eps=1e-05, affine=True)
            (1): SiLU()
            (2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (3): GroupNorm(32, 256, eps=1e-05, affine=True)
            (4): SiLU()
            (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (1): Sequential(
            (0): GroupNorm(32, 256, eps=1e-05, affine=True)
            (1): SiLU()
            (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (3): GroupNorm(32, 256, eps=1e-05, affine=True)
            (4): SiLU()
            (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (residual_input_conv): Modul

In [3]:
bgen = gen_bgen(train=False)
dataset = ERA5Dataset(bgen)

sample = dataset[100].unsqueeze(0)

In [4]:
with torch.no_grad():
    sample = sample.to(vae.device)

    rec, posterior = vae.forward(sample)

In [5]:
g_loss, log = loss.forward(
    inputs=sample,
    reconstructions=rec,
    posteriors=posterior,
    global_step=20000,
    optimizer="Generator"
)

In [6]:
print(log)

{'total_loss': tensor(0.2451, grad_fn=<AddBackward0>), 'rec_loss': tensor(0.2259), 'kl_loss': tensor(0.0152), 'g_loss': tensor(0.0039, grad_fn=<MulBackward0>)}


In [7]:
d_loss, log = loss.forward(
    inputs=sample,
    reconstructions=rec,
    posteriors=posterior,
    global_step=20000,
    optimizer="Discriminator"
)

In [8]:
print(log)

{'d_loss': tensor(0.7145, grad_fn=<MulBackward0>)}
